In [1]:
from utils.video_dataset import VideoFrameDataset
from config.torch_config import multiple_transform, device
from config.dataset import get_dataset_path


In [2]:
NUM_SEGMENTS = 3
FRAMES_PER_SEGMENT = 8

In [3]:
data_path, model_path = get_dataset_path()


In [4]:
dataset = VideoFrameDataset(
    root_path=data_path,
    transform=multiple_transform,
    num_segments=NUM_SEGMENTS,
    frames_per_segment=FRAMES_PER_SEGMENT,
)

classes = dataset.classes


In [5]:
from utils.loader import split_dataset


In [6]:
train_loader, test_loader = split_dataset(dataset, train_split=0.7, batch_size=64)


In [7]:
from utils.balance import check_balance_status

In [8]:
# check_balance_status(test_loader, classes)
# check_balance_status(train_loader, classes)


In [9]:
from lib.model import create_model

In [10]:
model = create_model(num_classes=len(classes))


In [11]:
from lib.train import train_model

In [12]:
train_model(model, train_loader, device, learning_rate=0.001, num_epochs=10)


Training on device: cuda


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 5-dimensional input of size [24, 24, 3, 224, 224] instead